# `TIME inc_ref` doesn't work with a register argument
This is a new problem in core revision 18 and the corresponding version of the assembler.

In [1]:
# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

# for low-level tests
from qick.asm_v2 import QickProgramV2

In [4]:
from qick.pyro import make_proxy
# soc, soccfg = make_proxy(ns_host="pynq4x2.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
# soc, soccfg = make_proxy(ns_host="pynq216.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
soc, soccfg = make_proxy(ns_host="pynq111-2.dhcp.fnal.gov", ns_port=8888, proxy_name="myqick")
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_ebfa326d304549db8e6830dbd841e1a9@131.225.82.18:35037

QICK configuration:

	Board: ZCU111

	Software version: 0.2.260
	Firmware timestamp: Wed Apr 17 20:55:00 2024

	Global clocks (MHz): tProcessor 384.000, RF reference 204.800

	8 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (10.667 us)
		fs=6144.000 MHz, fabric=384.000 MHz, 32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 0 is DAC228_T0_CH0 or RF board output 0
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (10.667 us)
		fs=6144.000 MHz, fabric=384.000 MHz, 32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 1 is DAC228_T0_CH1 or RF board output 1
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.333 us)
		fs=6144.000 MHz, fabric=384.000 MHz, 32-bit DDS, range=6144.000 MHz
		DAC tile 0, blk 2 is DAC228_T0_CH2 or RF board output 2
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.333 us)
		fs=6144.000 MHz, fabric

### using asm_v2
I have a minimal test program which plays a pulse on PMOD0_0 in a loop. There should be 10 pulses, separated by 2 us.

Here are two versions of the program. The first version uses a literal argument to `TIME inc_ref`, and this works. The second version uses a register argument, and this does not work: there is almost no gap between pulses, it's like the reference time did not change.

I checked that the binary output from the assembler did not change between 

In [65]:
# this works
reps = 10

prog = QickProgramV2(soccfg)
# initial delay for setup
prog.delay(1.0)
prog.open_loop(reps, "myloop")

prog.trigger(pins=[0], t=0)
prog.delay(1.0)
# prog.delay(QickSweep1D("myloop", 1.0, 1.0))

prog.close_loop()
prog.end()

print("text ASM:")
print(prog.asm())
# print("machine code:")
# prog.compile()
# for a in prog.binprog['pmem']: print([hex(b) for b in a])
# for x in prog.prog_list: print(x)

prog.run(soc)

text ASM:
     TIME ##384 inc_ref 
     REG_WR r0 imm ##10 
MYLOOP:
     REG_WR s14 imm ##0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm ##10 
     DPORT_WR p0 imm 0 
     TIME ##384 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     JUMP HERE 



In [66]:
# this doesn't work
reps = 10

prog = QickProgramV2(soccfg)
# initial delay for setup
prog.delay(1.0)
prog.open_loop(reps, "myloop")

prog.trigger(pins=[0], t=0)
# prog.delay(1.0)
prog.delay(QickSweep1D("myloop", 1.0, 1.0))

prog.close_loop()
prog.end()

print("text ASM:")
print(prog.asm())
# print("machine code:")
# prog.compile()
# for a in prog.binprog['pmem']: print([hex(b) for b in a])
# for x in prog.prog_list: print(x)

prog.run(soc)

text ASM:
     REG_WR r1 imm ##384 
     TIME ##384 inc_ref 
     REG_WR r0 imm ##10 
MYLOOP:
     REG_WR s14 imm ##0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm ##10 
     DPORT_WR p0 imm 0 
     TIME inc_ref r1 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     JUMP HERE 



### only using the assembler
Just to make sure, I do the same test with the same two programs, but using the text ASM and the assembler, nothing else. Same results.

In [70]:
# this works
from qick.tprocv2_assembler import Assembler
pstr = """
     TIME #384 inc_ref 
     REG_WR r0 imm #10 
MYLOOP:
     REG_WR s14 imm #0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm #10 
     DPORT_WR p0 imm 0 
     TIME #384 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     .END"""
print("text ASM:")
print(pstr)

# # convert text ASM to machine code
# print("\nmachine code:")
# for a in Assembler.str_asm2bin(pstr)[1]:
#     print([hex(b) for b in a])

# # convert text ASM to ASM dicts
# print("\nASM dicts:")
# plist,labels = Assembler.str_asm2list(pstr)
# for a in plist:
#     print(a)
# for a in labels.items():
#     print(a)

# # convert ASM dicts back to text ASM
# print("\ntext ASM -> ASM dicts -> text ASM:")
# print(Assembler.list2asm(plist, labels))

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     TIME #384 inc_ref 
     REG_WR r0 imm #10 
MYLOOP:
     REG_WR s14 imm #0 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm #10 
     DPORT_WR p0 imm 0 
     TIME #384 inc_ref 
     REG_WR r0 op -op(r0 - #1) -uf 
     JUMP MYLOOP -if(NZ) 
     .END


In [72]:
# this doesn't work
from qick.tprocv2_assembler import Assembler
pstr = """
     REG_WR r1 imm #384
     TIME #384 inc_ref
     REG_WR r0 imm #10
MYLOOP:
     REG_WR s14 imm #0
     DPORT_WR p0 imm 1
     REG_WR s14 imm #10
     DPORT_WR p0 imm 0
     TIME inc_ref r1
     REG_WR r0 op -op(r0 - #1) -uf
     JUMP MYLOOP -if(NZ)
     .END"""
print("text ASM:")
print(pstr)

# # convert text ASM to machine code
# print("\nmachine code:")
# for a in Assembler.str_asm2bin(pstr)[1]:
#     print([hex(b) for b in a])

# # convert text ASM to ASM dicts
# print("\nASM dicts:")
# plist,labels = Assembler.str_asm2list(pstr)
# for a in plist:
#     print(a)
# for a in labels.items():
#     print(a)

# # convert ASM dicts back to text ASM
# print("\ntext ASM -> ASM dicts -> text ASM:")
# print(Assembler.list2asm(plist, labels))

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     REG_WR r1 imm #384
     TIME #384 inc_ref
     REG_WR r0 imm #10
MYLOOP:
     REG_WR s14 imm #0
     DPORT_WR p0 imm 1
     REG_WR s14 imm #10
     DPORT_WR p0 imm 0
     TIME inc_ref r1
     REG_WR r0 op -op(r0 - #1) -uf
     JUMP MYLOOP -if(NZ)
     .END


### simplify
Unroll the loop, just play 5 pulses. Same behavior.

In [90]:
# works
from qick.tprocv2_assembler import Assembler
pstr = """
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     .END"""
print("text ASM:")
print(pstr)

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref #384
     .END


In [87]:
# doesn't work
from qick.tprocv2_assembler import Assembler
pstr = """
     REG_WR r1 imm #384
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     .END"""
print("text ASM:")
print(pstr)

binprog = {'pmem': Assembler.str_asm2bin(pstr)[1], 'wmem': np.zeros((0,8), dtype=np.int32)}
soc.load_bin_program(binprog)
soc.start_tproc()

text ASM:

     REG_WR r1 imm #384
     TIME inc_ref #384
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     DPORT_WR p0 imm 1 @0
     DPORT_WR p0 imm 0 @10
     TIME inc_ref r1
     .END
